### Processa Decisões
1. Identifica o resultado (Improcedente, Procedente ou Procedente em Parte)
2. Atualiza os processos com as infos da decisão (Data, Resultado e Juiz)


In [1]:
# Importando as Bibliotecas
import pandas as pd
import json
import pymongo
import yaml

In [3]:
# Carrega Configurações do arquivo config
with open("../config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]

In [4]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]


In [5]:
# Coleções Mongo
col_processos = db['processo']
col_decisoes = db['decisoes']

In [6]:
#df = pd.DataFrame.from_records(processos,index=['nro_processo','link_processo','nome_parte'])
dados = pd.DataFrame(list(col_processos.find({})))
df_processos = pd.json_normalize(dados['processo'])
df_processos.shape

(48204, 8)

In [7]:
# Processos por empresa
df_processos.groupby('cod_empresa')['nro_processo'].count()

cod_empresa
ALLIANZ         4315
AZUL            5682
EMS               91
GOL             4195
MAPFRE          5281
MLUIZA          1775
N/I               38
PORTOSEGURO    14436
SULAMERICA      3317
TAM             6239
TOKIOMARINE     2832
Name: nro_processo, dtype: int64

# Decisões

In [8]:
# Carrega base de decisões total
dados_decisoes = pd.DataFrame(list(col_decisoes.find({})))
df_decisoes = pd.json_normalize(dados_decisoes['decisao'])

In [10]:
df_decisoes.shape

(13850, 10)

In [55]:
col_decisoes.count_documents({"decisao.resultado": {"$exists": False} })

3629

In [90]:
col_decisoes.count_documents(
    {'$and':[{'decisao.texto_decisao':
    {'$regex':"( EXTINTA a presente ação)"}},
    {"decisao.resultado": {"$exists": False} }]}
)

10

In [89]:
col_decisoes.find_one(
    {'$and':[
        {'decisao.texto_decisao': {'$regex':"(  )"}},
        {'decisao.texto_decisao': {'$regex':"( EXTINT)"}},
        {"decisao.resultado": {"$exists": False} },
        {"decisao.nro_processo": {"$ne": '1012201-38.2022.8.26.0001'} }]}
)


{'_id': ObjectId('632d490fbb23267a64779f0c'),
 'decisao': {'nro_processo': '1004112-02.2017.8.26.0001',
  'classe': 'Execução de Título Extrajudicial',
  'assunto': 'Fiança',
  'magistrado': 'Fernanda de Carvalho Queiroz',
  'comarca': 'SÃO PAULO',
  'foro': 'Foro Regional I - Santana',
  'vara': '4ª Vara Cível',
  'data_disp': '22/05/2022',
  'texto_decisao': 'TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\nCOMARCA de SÃO PAULO\n\n4ª Vara Cível\n\nAvenida Engenheiro Caetano Álvares,  594, 2º andar, salas 202 e 204, Casa  Verde - CEP 02546-000, Fone: (11)-3951-2525, São Paulo-SP - E-mail: santana4cv@tjsp.jus.br\n\n\n\nAutos nº 1004112-02.2017.8.26.0001 - p. \n\nSENTENÇA\n\n\n\nAção:\n\n1004112-02.2017.8.26.0001 - Execução de Título Extrajudicial\n\nExequente:\n\nPorto Seguro Companhia de Seguros Gerais\n\nExecutado:\n\nRosileine Inacio Alves\n\n\n\n\n\nVistos.\n\nF. 387 - Comprovou-se a integral satisfação do crédito exequendo.\n\nPosto isso, declaro EXTINTA a presente ação com fulcro no

### Tags "Parcialmente Procedente" 

In [31]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( PARCIALMENTE PROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Parcialmente Procedente'}}
)


col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( Parcialmente Procedente)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Parcialmente Procedente'}}
)

### "Tag Improcedente"

In [52]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( IMPROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Improcedente'}}
)

col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( IMPPROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Improcedente'}}
)

col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( Improcedente)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Improcedente'}}
)

In [54]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( improcedente)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Improcedente'}}
)

### Tag "Procedente"

In [49]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( PROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Procedente'}}
)

In [50]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( Procedente)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Procedente'}}
)

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( PROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Procedente'}}
)

### Acordo

In [60]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( acordo celebrado entre as partes )"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

In [63]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( acordo que chegaram as partes )"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

In [68]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( acordo a que chegaram as partes)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

In [ ]:
#o acordo ora celebrado entre as partes
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( acordo a que chegaram as partes)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

In [79]:
col_decisoes.update_many(
    {'$and':[
        {'decisao.texto_decisao': {'$regex':"( HOMOLOGO )"}},
        {'decisao.texto_decisao': {'$regex':"( JULGO EXTINTO )"}},
        {"decisao.resultado": {"$exists": False} }]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

### Extinto

In [88]:
col_decisoes.update_many(
    {'$and':[
        {'decisao.texto_decisao': {'$regex':"( JULGO EXTINT)"}},
        {"decisao.resultado": {"$exists": False} }]},
    {'$set':{'decisao.resultado':'Extinto'}}
)

In [91]:
col_decisoes.update_many(
    {'$and':[
        {'decisao.texto_decisao': {'$regex':"( EXTINTA a presente ação)"}},
        {"decisao.resultado": {"$exists": False} }]},
    {'$set':{'decisao.resultado':'Extinto'}}
)